In [1]:
from herbie import FastHerbie
import numpy as np
import pandas as pd
import xarray as xr
import time
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from toolbox import EasyMap, pc

In [2]:
# set search params
period = pd.date_range(
    start="2013-01-01",
    end="2013-01-01")

# Herbie : quick
tic = time.perf_counter()
H = FastHerbie(period, model="gefs_reforecast",fxx=[100,270], member=0, variable_level="hgt_pres_abv700mb")
toc = time.perf_counter()
print(f"finished FastHerbie in {toc - tic:0.4f} seconds. Now putting into xarray...")

finished FastHerbie in 1.4010 seconds. Now putting into xarray...


In [3]:
# xarray: takes ~15 minutes per month, look into cutting more?
ds = H.xarray(":500 mb:")
toc2 = time.perf_counter()
print(f"finished xarray in {(toc2 - toc)/60:0.4f} minutes. Now clipping and saving...")


finished xarray in 1.2555 minutes. Now clipping and saving...


In [4]:
steps = np.arange(168,360,12,dtype='timedelta64[ns]')*(1000000000*60*60)
lats = np.arange(45,65.5,0.5)[::-1]
lons = np.arange(230,250.5,0.5)
ds = ds.sel(latitude=lats,longitude=lons,step=steps)

In [5]:
ds = ds.gh.isel(step=1)

In [6]:
ax = EasyMap("50m",crs=ds.herbie.crs,figsize=[10,6]).STATES().BORDERS().ax
p = ax.pcolormesh(
    ds.longitude, ds.latitude, ds.gh, transform=pc, cmap="Greens"
)
plt.colorbar(
    p,
    ax=ax,
    orientation="horizontal",
    pad=0.01,
    shrink=0.8,
    label=f"{ds.gh.GRIB_name}({ds.gh.GRIB_units})",
)

# Add 500 hPa Geopotential height
ax.contour(
    ds.longitude,
    ds.latitude,
    ds.gh,
    colors="k",
    linewidths=0.5,
    levels=range(0, 10000, 120),
)

ax.set_title(
    f"{ds.model.upper()} Geopotential height\nValid: {ds.valid_time.dt.strftime('%H:%M UTC %d %b %Y').item()}",
    loc="left",
)
#ax.set_title(ds.gh.GRIB_name, loc="right")

AttributeError: 'DataArray' object has no attribute 'herbie'